In [1]:
#Import

#pandas
import pandas as pd
from pandas import Series,DataFrame

#numpy
import numpy as np

In [2]:
#路径
path='C:\Users\Wvv\Datacastle_subsidy'

#借书记录(数据集1)
borrow_test_name=['id','borrow_date','book_name','book_num']
borrow_test=pd.read_table(path+'/final_test/borrow_final_test.txt',sep=',',header=None,names=borrow_test_name)

#删除重复行并浏览前几行数据
borrow_test=borrow_test.drop_duplicates()
borrow_test.head()

,id,borrow_date,book_name,book_num
0,27604,2014/10/31,Visual C++精彩实例详解 Visual C++ jing cai shi li xi...,TP312C 535
1,19774,2013/9/9,Photoshop CS4 中文版完全自学攻略 Photoshop CS4 zhong we...,TP391.41 678.4
2,14482,2014/10/13,认知心理学 ren zhi xin li xue Cognitive psychology ...,B842.1 S837
3,26968,2014/2/28,电工原理的MATLAB实现 dian gong yuan li de MATLAB shi ...,NaN
4,22567,2015/6/12,2009年考研英语读真题记单词 2009 nian kao yan ying yu du z...,H313-42 695 2009


In [2]:
#路径
path='C:\Users\Wvv\Datacastle_subsidy'

#借书记录(数据集2)
borrow_test_name=['id','borrow_date','book_name','book_num']
borrow_test=pd.read_table(path+'/test/borrow_test.txt',sep=',',header=None,names=borrow_test_name)

#删除重复行并浏览前几行数据
borrow_test=borrow_test.drop_duplicates()
borrow_test.head()

,id,borrow_date,book_name,book_num
0,842,2013/12/23,人性论 ren xing lun 上 (英)休谟(David Hume)著 关文运译,B561.291 H963-2 V1
1,26615,2014/11/25,C语言程序设计基础教程 C yu yan cheng xu she ji ji chu ji...,TP312C-43 852.12
2,12762,2013/12/3,贾平凹散文选 jia ping ao san wen xuan / 贾平凹著,I267 146-13
3,11291,2015/4/24,概率论与数理统计 / 徐全智，吕恕[主编],O21-43 337-2 E2
4,9595,2014/11/20,非营利组织管理概论 fei ying li zu zhi guan li gai lun ...,C912.2-43 92


In [3]:
#删除无用列

borrow_test.drop(['book_name','book_num'],axis=1,inplace=True)

In [4]:
#定义time转化函数

from datetime import datetime
def time_transform(x):
    return datetime.strptime(x,'%Y/%m/%d')

In [5]:
#转换字符串型的time成时间戳格式
borrow_test['borrow_date']=borrow_test['borrow_date'].apply(time_transform)

In [6]:
#时间格式转换

borrow_test['month']=borrow_test['borrow_date'].apply(lambda x:x.strftime('%m'))
borrow_test['weekday']=borrow_test['borrow_date'].apply(lambda x:x.strftime('%A'))

In [7]:
#month
b_month=borrow_test.groupby('month')['month'].count()
#对month进行one-hot编码

month_dummies=pd.get_dummies(borrow_test['month'])
month_dummies.columns=['borrow_month'+x for x in list(b_month.index)]

#连接两表

borrow_test=borrow_test.join(month_dummies)

In [8]:
#weekday（周末与非周末）

def get_week(week):
    a=['Monday','Tuesday','Wednesday','Thursday','Friday']
    #b=['Saturday','Sunday']
    if week in a:
        return 'wd'
    else:
        return 'vd'

borrow_test['weekday']=borrow_test['weekday'].apply(get_week)

week_dummies=pd.get_dummies(borrow_test['weekday'])
week_dummies.columns=['borrow_resting_day','borrow_working_day']

borrow_test=borrow_test.join(week_dummies)

In [9]:
#根据id聚合求解各个特征的产生的总数

count_columns=list(borrow_test.columns)[4:]

#count_columns
count_pivot=borrow_test.pivot_table(index='id',values=count_columns,aggfunc=np.count_nonzero)

In [10]:
#每个id借书的总次数

borrow_test_feature=DataFrame(borrow_test.groupby('id')['id'].count())
borrow_test_feature.columns=['borrow_sum']

#连接表
borrow_test_feature=borrow_test_feature.join(count_pivot)

In [11]:
#周末与非周末借书占比

borrow_week_rate=borrow_test.pivot_table(index='id',values=['borrow_resting_day','borrow_working_day'],aggfunc=np.mean)
borrow_week_rate.columns=['borrow_resting_day_rate','borrow_working_day_rate']

#连接表
borrow_test_feature=borrow_test_feature.join(borrow_week_rate)

In [12]:
#每个id活跃天数

borrow_day=DataFrame(borrow_test.groupby(['id','borrow_date'])['id'].count())
borrow_day.to_csv('borrow_input/borrow_test_day.csv',index=True)
borrow_day_count=pd.read_csv('borrow_input/borrow_test_day.csv')
borrow_day_count=DataFrame(borrow_day_count.groupby('id')['id.1'].count())
borrow_day_count.columns=['borrow_day_sum']

#连接表
borrow_test_feature=borrow_test_feature.join(borrow_day_count)

In [13]:
#每天平均借书量

borrow_test_feature['borrow_day_per']=borrow_test_feature['borrow_sum'].divide(borrow_test_feature['borrow_day_sum'],fill_value=0)

In [14]:
borrow_test_feature.to_csv('borrow_input/borrow_test_feature_f.csv',index='id')

In [15]:
borrow_test_feature.head()

,borrow_sum,borrow_month01,borrow_month02,borrow_month03,borrow_month04,borrow_month05,borrow_month06,borrow_month07,borrow_month08,borrow_month09,borrow_month10,borrow_month11,borrow_month12,borrow_resting_day,borrow_working_day,borrow_resting_day_rate,borrow_working_day_rate,borrow_day_sum,borrow_day_per
id,,,,,,,,,,,,,,,,,,,
32,13,0,2,5,6,0,0,0,0,0,0,0,0,3,10,0.230769,0.769231,6,2.166667
38,19,2,1,3,2,0,2,4,0,1,2,0,2,4,15,0.210526,0.789474,10,1.900000
39,53,7,0,18,2,2,0,0,2,12,4,6,0,20,33,0.377358,0.622642,15,3.533333
47,2,0,0,0,1,0,0,0,0,0,1,0,0,1,1,0.500000,0.500000,2,1.000000
48,14,0,0,3,4,0,6,0,0,1,0,0,0,4,10,0.285714,0.714286,4,3.500000


In [16]:
len(borrow_test_feature)

7158